取得 Account ID 驗證 AWS 本機設置正確完成

In [33]:
import boto3

# 載入 .env 檔案中的環境變數
from dotenv import load_dotenv
import os
load_dotenv()

# 建立 STS 客戶端
sts_client = boto3.client("sts")

# 呼叫 get_caller_identity 以取得當前 AWS 帳號資訊
response = sts_client.get_caller_identity()
account_id = response["Account"]

print(f"AWS Account ID: {account_id}")

AWS Account ID: 145745076948


遍歷所有 S3 bucket 中的資源並顯示樹狀結構

In [2]:
import boto3

# 建立 S3 資源對象
s3 = boto3.resource('s3')

def list_s3_bucket_structure(bucket_name):
    bucket = s3.Bucket(bucket_name)
    print(f"Bucket: {bucket_name}")
    
    has_objects = False
    for obj in bucket.objects.all():
        print(" └── ", obj.key)
        has_objects = True

    if not has_objects:
        print(" └── <此 bucket 為空>")

# 取得所有 bucket 名稱並顯示每個 bucket 的樹狀結構
s3_client = boto3.client("s3")
buckets = s3_client.list_buckets()["Buckets"]

if buckets:
    for bucket in buckets:
        list_s3_bucket_structure(bucket["Name"])
else:
    print("當前為空的：沒有任何 S3 bucket")

Bucket: c133864a3391506l8170626t1w14574507694-lambdabucket-6tapr19qkhoa
 └──  s3cleaner_function.zip
Bucket: c133864a3391506l8170626t1w145745076948-labbucket-ka1fhe2jobcp
 └──  lab71/challenge/sample.mp4
 └──  lab71/polly-sample/es.test.txt
 └──  lab71/transcribe-sample/test.wav
 └──  lab71/translate-sample/test.txt


In [ ]:
# 建立 S3 客戶端
s3_client = boto3.client("s3")

# 列出 Buckets
def list_buckets():
    try:
        # 呼叫 list_buckets 方法以取得所有 bucket
        response = s3_client.list_buckets()

        # 從回應中提取每個 bucket 名稱並篩選包含 'labbucket' 的
        bucket_names = [
            bucket["Name"]
            for bucket in response["Buckets"]
            if "labbucket" in bucket["Name"]
        ]
        return bucket_names
    except Exception as e:
        print(f"無法取得 bucket 列表。錯誤訊息：{e}")
        return None


# 取得篩選後的 bucket 名稱
bucket_names = list_buckets()
# 顯示取得的 bucket 名稱
if bucket_names:
    print("包含 'labbucket' 的 Bucket 名稱列表：")
    for name in bucket_names:
        print(name)
else:
    print("沒有找到包含 'labbucket' 的 Bucket。")

包含 'labbucket' 的 Bucket 名稱列表：
c133864a3391506l8170626t1w145745076948-labbucket-ka1fhe2jobcp


列出 S3 bucket 中的所有視頻文件

In [3]:
import boto3

s3_client = boto3.client("s3")
bucket_name = "aws-tc-largeobjects"
prefix = "CUR-TF-200-ACMNLP-1/video/"

response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
for item in response.get("Contents", []):
    print(item["Key"])


CUR-TF-200-ACMNLP-1/video/Mod01_Course Overview.mp4
CUR-TF-200-ACMNLP-1/video/Mod02_Intro.mp4
CUR-TF-200-ACMNLP-1/video/Mod02_Sect01.mp4
CUR-TF-200-ACMNLP-1/video/Mod02_Sect02.mp4
CUR-TF-200-ACMNLP-1/video/Mod02_Sect03.mp4
CUR-TF-200-ACMNLP-1/video/Mod02_Sect04.mp4
CUR-TF-200-ACMNLP-1/video/Mod02_Sect05.mp4
CUR-TF-200-ACMNLP-1/video/Mod02_WrapUp.mp4
CUR-TF-200-ACMNLP-1/video/Mod03_Intro.mp4
CUR-TF-200-ACMNLP-1/video/Mod03_Sect01.mp4
CUR-TF-200-ACMNLP-1/video/Mod03_Sect02_part1.mp4
CUR-TF-200-ACMNLP-1/video/Mod03_Sect02_part2.mp4
CUR-TF-200-ACMNLP-1/video/Mod03_Sect02_part3.mp4
CUR-TF-200-ACMNLP-1/video/Mod03_Sect03_part1.mp4
CUR-TF-200-ACMNLP-1/video/Mod03_Sect03_part2.mp4
CUR-TF-200-ACMNLP-1/video/Mod03_Sect03_part3.mp4
CUR-TF-200-ACMNLP-1/video/Mod03_Sect04_part1.mp4
CUR-TF-200-ACMNLP-1/video/Mod03_Sect04_part2.mp4
CUR-TF-200-ACMNLP-1/video/Mod03_Sect04_part3.mp4
CUR-TF-200-ACMNLP-1/video/Mod03_Sect05.mp4
CUR-TF-200-ACMNLP-1/video/Mod03_Sect06.mp4
CUR-TF-200-ACMNLP-1/video/Mod03_Sect

取得 target_bucket_name

In [7]:
import boto3
import os
from botocore.exceptions import ClientError

# 設定 S3 客戶端
s3_client = boto3.client("s3")

# 列出符合條件的 Buckets
def list_labbuckets():
    try:
        response = s3_client.list_buckets()
        bucket_names = [
            bucket["Name"]
            for bucket in response["Buckets"]
            if "labbucket" in bucket["Name"]
        ]
        return bucket_names
    except Exception as e:
        print(f"無法取得 bucket 列表。錯誤訊息：{e}")
        return None

# 獲取包含 'labbucket' 的第一個 bucket 名稱
bucket_names = list_labbuckets()
target_bucket_name = bucket_names[0] if bucket_names else None

# 顯示 bucket 名稱，並確認找到符合條件的 bucket
if target_bucket_name:
    print(f"使用的 Bucket: {target_bucket_name}")
else:
    print("沒有找到包含 'labbucket' 的 Bucket。")
    exit()

使用的 Bucket: c133864a3391506l8170626t1w145745076948-labbucket-ka1fhe2jobcp


步驟 1：取得第一個 .mp4 文件名稱

In [8]:
source_bucket_name = "aws-tc-largeobjects"
source_prefix = "CUR-TF-200-ACMNLP-1/video/"
try:
    response = s3_client.list_objects_v2(
        Bucket=source_bucket_name,
        Prefix=source_prefix
    )
    first_mp4_file = None
    for item in response.get("Contents", []):
        if item["Key"].endswith(".mp4"):
            first_mp4_file = item["Key"]
            print(f"找到第一個 mp4 文件: {first_mp4_file}")
            break
    if not first_mp4_file:
        print("未找到任何 mp4 文件。")
except ClientError as e:
    print("無法訪問 S3 存儲桶。錯誤訊息:", e)
    first_mp4_file = None

找到第一個 mp4 文件: CUR-TF-200-ACMNLP-1/video/Mod01_Course Overview.mp4


步驟 2：下載文件至本地

In [ ]:
if first_mp4_file:
    # 取得文件名稱
    local_file_name = os.path.basename(first_mp4_file)
    try:
        s3_client.download_file(
            source_bucket_name,
            first_mp4_file,
            local_file_name
        )
        print(f"文件下載成功: {local_file_name}")
    except ClientError as e:
        print("文件下載失敗。錯誤訊息:", e)
else:
    print("無法找到第一個 mp4 文件，無法執行下載和上傳操作。")

文件下載成功: Mod01_Course Overview.mp4


步驟 3：上傳文件至目標 S3 bucket 的 mp4 資料夾

In [10]:
# 檢查是否已下載文件並確保目標 bucket 已存在
if local_file_name and target_bucket_name:
    # 目標 S3 路徑
    target_key = f"mp4/{local_file_name}"
    try:
        s3_client.upload_file(local_file_name, target_bucket_name, target_key)
        print(f"文件 {local_file_name} 成功上傳至 {target_bucket_name}/{target_key}")
    except ClientError as e:
        print("文件上傳失敗。錯誤訊息:", e)
else:
    print("無法上傳，請確認文件已下載或 bucket 設定正確。")


文件 Mod01_Course Overview.mp4 成功上傳至 c133864a3391506l8170626t1w145745076948-labbucket-ka1fhe2jobcp/mp4/Mod01_Course Overview.mp4


進行轉錄

In [21]:
import boto3
import uuid

# 目標 bucket
bucket_name = target_bucket_name
first_mp4_file_uri = \
    f"s3://{target_bucket_name}/mp4/{local_file_name}"

# 設定 AWS Transcribe 客戶端
transcribe_client = boto3.client("transcribe")

# 生成唯一的轉錄工作名稱
job_uuid = uuid.uuid1()
transcribe_job_name = f"transcribe-job-{job_uuid}"
# 自訂義轉錄輸出的檔案名稱
transcribe_output_filename = "transcribe_output.txt"
# 定義轉錄輸出的檔案名稱和路徑
transcribe_output_key = \
    f"capstone_project/{transcribe_output_filename}"

# 啟動轉錄工作
try:
    response = transcribe_client.start_transcription_job(
        TranscriptionJobName=transcribe_job_name,
        Media={"MediaFileUri": first_mp4_file_uri},
        MediaFormat="mp4",
        LanguageCode="en-US",
        OutputBucketName=bucket_name,
        OutputKey=transcribe_output_key,
    )
    print("轉錄作業已啟動:", response)
except Exception as e:
    print("無法啟動轉錄作業。錯誤訊息:", e)


轉錄作業已啟動: {'TranscriptionJob': {'TranscriptionJobName': 'transcribe-job-24a1531a-9884-11ef-a5ab-3a328b8844c7', 'TranscriptionJobStatus': 'IN_PROGRESS', 'LanguageCode': 'en-US', 'MediaFormat': 'mp4', 'Media': {'MediaFileUri': 's3://c133864a3391506l8170626t1w145745076948-labbucket-ka1fhe2jobcp/mp4/Mod01_Course Overview.mp4'}, 'StartTime': datetime.datetime(2024, 11, 2, 3, 4, 39, 628000, tzinfo=tzlocal()), 'CreationTime': datetime.datetime(2024, 11, 2, 3, 4, 39, 607000, tzinfo=tzlocal())}, 'ResponseMetadata': {'RequestId': 'bb0c2188-3a2e-4d3b-bbbe-faa2c4e27492', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'bb0c2188-3a2e-4d3b-bbbe-faa2c4e27492', 'content-type': 'application/x-amz-json-1.1', 'content-length': '366', 'date': 'Fri, 01 Nov 2024 19:04:39 GMT'}, 'RetryAttempts': 0}}


檢查回傳值中 transcription job 的狀態，直到顯示 COMPLETED 或 FAILED 為止。

In [22]:
import boto3
from time import sleep

# 初始化 AWS Transcribe 客戶端
transcribe_client = boto3.client("transcribe")

# 假設前面已經定義了 transcribe_job_name 變數
job = None
while True:
    # 檢查轉錄工作狀態，並將結果存放在變數 job 中
    try:
        job = transcribe_client.get_transcription_job(
            TranscriptionJobName=transcribe_job_name
        )
    except Exception as e:
        print("無法檢查轉錄作業狀態。錯誤訊息:", e)
        break

    # 取得當前轉錄作業的狀態
    job_status = job["TranscriptionJob"]["TranscriptionJobStatus"]

    # 如果轉錄作業已完成或失敗則跳出循環
    if job_status in ["COMPLETED", "FAILED"]:
        print(f"\n轉錄作業狀態：{job_status}")
        break

    print(".", end="", flush=True)
    # 每 20 秒檢查一次狀態
    sleep(20)

# 確認轉錄結果
if job_status == "COMPLETED":
    transcription_file_uri = job["TranscriptionJob"]["Transcript"]["TranscriptFileUri"]
    print("\n轉錄作業已完成。轉錄文件的 URI:", transcription_file_uri)
else:
    print("\n轉錄作業失敗。")
    if "FailureReason" in job["TranscriptionJob"]:
        print("失敗原因:", job["TranscriptionJob"]["FailureReason"])


.
轉錄作業狀態：COMPLETED

轉錄作業已完成。轉錄文件的 URI: https://s3.us-east-1.amazonaws.com/c133864a3391506l8170626t1w145745076948-labbucket-ka1fhe2jobcp/capstone_project/transcribe_output.txt


In [23]:
job

{'TranscriptionJob': {'TranscriptionJobName': 'transcribe-job-24a1531a-9884-11ef-a5ab-3a328b8844c7',
  'TranscriptionJobStatus': 'COMPLETED',
  'LanguageCode': 'en-US',
  'MediaSampleRateHertz': 48000,
  'MediaFormat': 'mp4',
  'Media': {'MediaFileUri': 's3://c133864a3391506l8170626t1w145745076948-labbucket-ka1fhe2jobcp/mp4/Mod01_Course Overview.mp4'},
  'Transcript': {'TranscriptFileUri': 'https://s3.us-east-1.amazonaws.com/c133864a3391506l8170626t1w145745076948-labbucket-ka1fhe2jobcp/capstone_project/transcribe_output.txt'},
  'StartTime': datetime.datetime(2024, 11, 2, 3, 4, 39, 628000, tzinfo=tzlocal()),
  'CreationTime': datetime.datetime(2024, 11, 2, 3, 4, 39, 607000, tzinfo=tzlocal()),
  'CompletionTime': datetime.datetime(2024, 11, 2, 3, 5, 24, 902000, tzinfo=tzlocal()),
  'Settings': {'ChannelIdentification': False, 'ShowAlternatives': False}},
 'ResponseMetadata': {'RequestId': 'cc223f4b-ec88-4f1f-9755-34fee84db5cc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requesti

取得輸出的路徑

In [24]:
transcription_file = job["TranscriptionJob"]["Transcript"]["TranscriptFileUri"]
print(transcription_file)

https://s3.us-east-1.amazonaws.com/c133864a3391506l8170626t1w145745076948-labbucket-ka1fhe2jobcp/capstone_project/transcribe_output.txt


In [25]:
import boto3

# 初始化 S3 客戶端
s3_client = boto3.client("s3")

# 假設 bucket_name 與 transcribe_output_key 已定義
# 並且 transcribe_output_filename 已定義為本地存儲的名稱

try:
    with open(transcribe_output_filename, "wb") as f:
        # 使用下載文件的完整 S3 路徑
        s3_client.download_fileobj(
            bucket_name,  # bucket 名稱
            transcribe_output_key,  # 完整的 S3 文件路徑
            f  # 本地文件路徑
        )
    print(f"文件 {transcribe_output_filename} 已成功下載。")
except Exception as e:
    print(f"無法下載文件。錯誤訊息：{e}")


文件 transcribe_output.txt 已成功下載。


### 標準化文本

移除標點符號與特殊字符：使用正則表達式刪除不需要的符號，保留必要的語句結構。

轉換成小寫：這可以統一格式，便於後續的關鍵詞提取。

刪除停用詞：如「the」、「and」等無法提供語義的詞。

分句或分段：根據需求，可以將文本按句子或段落進行切分，以便於進一步處理。

In [ ]:
import json
import re
from nltk.corpus import stopwords

# 讀取轉錄文本文件
with open("transcribe_output.txt", "r") as file:
    # 使用 json.load 來解析 JSON 格式
    transcript_data = json.load(file)

# 提取文本內容
text = transcript_data["results"]["transcripts"][0]["transcript"]

# 1. 移除標點符號和特殊字符
text = re.sub(r'[^\w\s]', '', text)

# 2. 轉換為小寫
text = text.lower()

# 3. 移除停用詞 (使用 NLTK 的停用詞庫)
stop_words = set(stopwords.words("english"))
text = " ".join([word for word in text.split() if word not in stop_words])

# 4. 分句或分段
sentences = text.split('.')

# 將標準化的文本保存到新的文件中
normalized_filename = "transcribe_output_std.txt"
with open(normalized_filename, "w") as normalized_file:
    normalized_file.write("\n".join(sentences))

print("標準化後的文本已保存至：", normalized_filename)


標準化後的文本已保存至: transcribe_output_std.txt


檢查 comprehend 權限

In [31]:
import boto3

# 建立 IAM 客戶端
iam_client = boto3.client('iam')

# 當前使用者的 ARN
user_arn = 'arn:aws:sts::145745076948:assumed-role/voclabs/user2069478=_________'

# 欲檢查的操作
actions = [
    "comprehend:DetectKeyPhrases",
    "comprehend:DetectEntities"
]

# 執行權限模擬
try:
    response = iam_client.simulate_principal_policy(
        PolicySourceArn=user_arn,
        ActionNames=actions
    )

    # 輸出結果
    for result in response['EvaluationResults']:
        print(f"操作: {result['EvalActionName']}")
        print(f"授權狀態: {result['EvalDecision']}")
        if 'EvalDecisionDetails' in result:
            print(f"詳細說明: {result['EvalDecisionDetails']}")
        print("-" * 30)

except Exception as e:
    print(f"無法查詢權限，錯誤訊息: {e}")


無法查詢權限，錯誤訊息: An error occurred (InvalidInput) when calling the SimulatePrincipalPolicy operation: Invalid Entity Arn: arn:aws:sts::145745076948:assumed-role/voclabs/user2069478=_________ does not clearly define entity type and name.


檢查當前使用者的已附加策略

In [32]:
# 檢查 IAM 使用者策略
try:
    user_policies = iam_client.list_attached_user_policies(UserName='user2069478')
    for policy in user_policies['AttachedPolicies']:
        print(f"策略名稱: {policy['PolicyName']}")
        print(f"策略 ARN: {policy['PolicyArn']}")
except Exception as e:
    print(f"無法取得使用者的策略列表。錯誤訊息：{e}")


無法取得使用者的策略列表。錯誤訊息：An error occurred (NoSuchEntity) when calling the ListAttachedUserPolicies operation: The user with name user2069478 cannot be found.


# 轉換 Lab

In [30]:
import boto3
import json

# 設定 AWS Comprehend 客戶端
comprehend_client = boto3.client("comprehend")

# 讀取標準化後的文本內容
normalized_file_path = "transcribe_output_std.txt"
with open(normalized_file_path, "r") as file:
    text = file.read()

# 確認文本長度，Comprehend 對單次請求文本的長度限制是 5000 字符
max_length = 5000
if len(text) > max_length:
    text = text[:max_length]  # 只提取前 5000 字符進行分析

# 1. 提取關鍵詞組
key_phrases = []
try:
    key_phrases_response = comprehend_client.detect_key_phrases(
        Text=text,
        LanguageCode="en"
    )
    key_phrases = key_phrases_response.get("KeyPhrases", [])
    print("關鍵詞組提取結果:")
    for phrase in key_phrases:
        print(f"- {phrase['Text']} (信心分數: {phrase['Score']:.2f})")
except Exception as e:
    print("提取關鍵詞組時出錯:", e)

# 2. 主題檢測（僅適用於文本中檢測具體實體）
entities = []
try:
    entities_response = comprehend_client.detect_entities(
        Text=text,
        LanguageCode="en"
    )
    entities = entities_response.get("Entities", [])
    print("\n主題檢測結果:")
    for entity in entities:
        print(f"- {entity['Text']} (類型: {entity['Type']}, 信心分數: {entity['Score']:.2f})")
except Exception as e:
    print("主題檢測時出錯:", e)

# 3. 將結果保存到新文件中
output_file_path = "comprehend_analysis_result.json"
analysis_results = {
    "KeyPhrases": key_phrases,
    "Entities": entities
}
with open(output_file_path, "w") as output_file:
    json.dump(analysis_results, output_file, ensure_ascii=False, indent=4)

print("\n分析結果已保存至:", output_file_path)


提取關鍵詞組時出錯: An error occurred (AccessDeniedException) when calling the DetectKeyPhrases operation: User: arn:aws:sts::145745076948:assumed-role/voclabs/user2069478=_________ is not authorized to perform: comprehend:DetectKeyPhrases because no identity-based policy allows the comprehend:DetectKeyPhrases action
主題檢測時出錯: An error occurred (AccessDeniedException) when calling the DetectEntities operation: User: arn:aws:sts::145745076948:assumed-role/voclabs/user2069478=_________ is not authorized to perform: comprehend:DetectEntities because no identity-based policy allows the comprehend:DetectEntities action

分析結果已保存至: comprehend_analysis_result.json
